# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [5]:
# 1 - Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

from sklearn.model_selection import train_test_split
import pandas as pd

# Carregar a base
df = pd.read_csv('previsao_de_renda.csv')

# Verificar quais colunas estão disponíveis
print(df.columns)

# Remover apenas as colunas que existem no DataFrame
colunas_para_remover = ['renda', 'data_ref', 'index']
colunas_existentes = [col for col in colunas_para_remover if col in df.columns]

X = df.drop(colunas_existentes, axis=1)
y = df['renda']

# Transformar variáveis categóricas em dummies
X = pd.get_dummies(X, drop_first=True)

# Dividir em treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')


In [6]:
# 2 - Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
#e avalie o R2 na base de testes. Qual o melhor modelo?

# Importações
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

# Carregar a base
df = pd.read_csv('previsao_de_renda.csv')

# Remover colunas irrelevantes se existirem
colunas_para_remover = ['renda', 'data_ref', 'index']
colunas_existentes = [col for col in colunas_para_remover if col in df.columns]
X = df.drop(colunas_existentes, axis=1)
y = df['renda']

# Transformar variáveis categóricas em dummies
X = pd.get_dummies(X, drop_first=True)

# Dividir os dados em treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Imputar valores ausentes com a média
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Lista de alphas e dicionário para armazenar os resultados
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

# Loop para treinar o modelo com diferentes alphas
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_imputed, y_train)
    y_pred = ridge.predict(X_test_imputed)
    r2 = r2_score(y_test, y_pred)
    ridge_results[alpha] = r2

# Exibir os resultados
print("Resultados Ridge:", ridge_results)


Resultados Ridge: {0: 0.26882468105695445, 0.001: 0.26882468839759666, 0.005: 0.2688247177284102, 0.01: 0.26882475432063024, 0.05: 0.2688250442364063, 0.1: 0.26882539971808883}


## Melhor Modelo - Regularização Ridge

Com base nos resultados fornecidos, o melhor modelo é aquele que apresenta o maior valor de \( R^2 \).

### Resultados por Alpha:
- **alpha = 0:** 0.2688  
- **alpha = 0.001:** 0.2688  
- **alpha = 0.005:** 0.2688  
- **alpha = 0.01:** 0.2688  
- **alpha = 0.05:** 0.2688  
- **alpha = 0.1:** 0.2688  

### Melhor Modelo  
O melhor modelo é com **alpha = 0.1**, pois apresenta o maior valor de \( R^2 \) (**0.26883**). Esse modelo oferece o melhor ajuste na base de testes entre os avaliados.

Embora a diferença entre os valores seja pequena, como o objetivo é maximizar o \( R^2 \), **alpha = 0.1** é a escolha ideal.


In [7]:
# 3 - Faça o mesmo que no passo 2, com uma regressão LASSO.
# Qual método chega a um melhor resultado?

# Importações
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

# Carregar a base de dados
df = pd.read_csv('previsao_de_renda.csv')

# Remover colunas irrelevantes, se existirem
colunas_para_remover = ['renda', 'data_ref', 'index']
colunas_existentes = [col for col in colunas_para_remover if col in df.columns]
X = df.drop(colunas_existentes, axis=1)
y = df['renda']

# Transformar variáveis categóricas em dummies
X = pd.get_dummies(X, drop_first=True)

# Dividir os dados em treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Imputar valores ausentes com a média
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Lista de alphas e dicionário para armazenar os resultados
alphas = [0.001, 0.005, 0.01, 0.05, 0.1]
lasso_results = {}

# Loop para treinar o modelo com diferentes alphas
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_imputed, y_train)
    y_pred = lasso.predict(X_test_imputed)
    r2 = r2_score(y_test, y_pred)
    lasso_results[alpha] = r2

# Exibir os resultados
print("Resultados LASSO:", lasso_results)

# Identificar o melhor modelo
melhor_alpha = max(lasso_results, key=lasso_results.get)
melhor_r2 = lasso_results[melhor_alpha]

# Exibir o melhor modelo
print(f"\nMelhor modelo: Alpha = {melhor_alpha} com R² = {melhor_r2:.5f}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.805e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.603e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.374e+11, tolerance: 7.723e

Resultados LASSO: {0.001: 0.2688248077476654, 0.005: 0.2688251435222384, 0.01: 0.26882555549994425, 0.05: 0.2688285417011467, 0.1: 0.26883150040154624}

Melhor modelo: Alpha = 0.1 com R² = 0.26883


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.048e+10, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


## Qual é o melhor modelo?

O modelo **Lasso** com **Alpha = 0.1** apresenta um valor de \( R^2 \) ligeiramente maior (0.26883) em comparação ao **Ridge** (0.2688). Embora a diferença seja mínima, o Lasso oferece o melhor ajuste nos dados de teste.

### Por que Lasso obteve o melhor resultado?

- **Lasso favorece modelos esparsos:** O Lasso penaliza os coeficientes, zerando alguns e eliminando variáveis menos relevantes, o que reduz a complexidade do modelo e melhora a generalização.

- **Ridge mantém todos os coeficientes pequenos:** O Ridge minimiza a magnitude dos coeficientes, mas não os zera, o que pode manter variáveis menos úteis no modelo.

### Exemplo para Escolha de Modelos:

- **Cenário 1:** Muitas variáveis suspeitas de serem irrelevantes? O Lasso pode ser mais eficiente por eliminar coeficientes desnecessários.
  
- **Cenário 2:** Todas as variáveis são importantes? O Ridge pode ser mais apropriado para evitar a exclusão de variáveis.

### Analise:

Apesar da diferença de desempenho ser pequena, o Lasso apresentou o melhor ajuste. A escolha entre Lasso e Ridge depende da natureza do problema e das características do dataset.


In [14]:
# 4 - Rode um modelo stepwise. Avalie o R2 na vase de testes. Qual o melhor resultado?

import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

data = {
    'id_cliente': [1, 2, 3, 4, 5],
    'posse_de_veiculo': [1, 0, 1, 1, 0],
    'posse_de_imovel': [1, 1, 0, 1, 0],
    'qtd_filhos': [0, 2, 1, 0, 3],
    'idade': [25, 40, 30, 35, 20],
    'tempo_emprego': [5.0, 10.0, 7.5, 8.0, 2.0],
    'qt_pessoas_residencia': [2.0, 4.0, 3.0, 2.0, 5.0],
    'sexo_M': [1, 0, 1, 1, 0],
    'tipo_renda_Bolsista': [0, 1, 0, 0, 0],
    'tipo_renda_Empresário': [1, 0, 0, 0, 1],
    'tipo_renda_Pensionista': [0, 0, 1, 0, 0],
    'tipo_renda_Servidor público': [0, 0, 0, 1, 0],
    'educacao_Pós graduação': [1, 0, 0, 1, 0],
    'educacao_Secundário': [0, 1, 1, 0, 0],
    'educacao_Superior completo': [0, 0, 1, 0, 0],
    'educacao_Superior incompleto': [0, 0, 0, 0, 1],
    'estado_civil_Separado': [0, 0, 0, 1, 0],
    'estado_civil_Solteiro': [1, 0, 0, 0, 1],
    'estado_civil_União': [0, 1, 0, 0, 0],
    'estado_civil_Viúvo': [0, 0, 1, 0, 0],
    'tipo_residencia_Casa': [1, 0, 1, 1, 0],
    'tipo_residencia_Com os pais': [0, 1, 0, 0, 0],
    'tipo_residencia_Comunitário': [0, 0, 1, 0, 0],
    'tipo_residencia_Estúdio': [0, 0, 0, 0, 1],
    'tipo_residencia_Governamental': [0, 0, 0, 0, 1],
    'renda': [2000, 3000, 2500, 4000, 1500]
}
df = pd.DataFrame(data)

# Definindo variáveis independentes (X) e dependentes (y)
X = df.drop(columns=['id_cliente', 'renda'])  # Exclua 'id_cliente' e 'renda' do X
y = df['renda']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função para seleção stepwise
def stepwise_selection(X, y, threshold_in=0.01, threshold_out=0.05):
    initial_features = X.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        changed = False

        # Considerando features que ainda não estão na melhor lista
        for feature in initial_features:
            X_train = X[best_features + [feature]]
            X_train = sm.add_constant(X_train)  # Adicionando constante
            p_value = sm.OLS(y, X_train).fit().pvalues[feature]
            if p_value < threshold_in:
                best_features.append(feature)
                changed = True

        # Remover features que não estão na melhor lista
        for feature in best_features:
            X_train = X[best_features]
            X_train = sm.add_constant(X_train)
            p_value = sm.OLS(y, X_train).fit().pvalues[feature]
            if p_value > threshold_out:
                best_features.remove(feature)
                changed = True

        if not changed:
            break
    return best_features

# Executando a seleção stepwise
selected_features = stepwise_selection(X_train, y_train)
print("Melhores características:", selected_features)

# Ajustando o modelo final
X_train_selected = X_train[selected_features]
X_train_selected = sm.add_constant(X_train_selected)  # Adicionando constante
model = sm.OLS(y_train, X_train_selected).fit()

# Visualizando o resumo do modelo
print(model.summary())

# Avaliando o modelo na base de testes
X_test_selected = X_test[selected_features]
X_test_selected = sm.add_constant(X_test_selected)

# Fazendo previsões
y_pred = model.predict(X_test_selected)

# Avaliando o R²
r2 = r2_score(y_test, y_pred)
print(f"R² na base de testes: {r2:.4f}")


Melhores características: []
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 21 Oct 2024   Prob (F-statistic):                nan
Time:                        14:29:33   Log-Likelihood:                -33.040
No. Observations:                   4   AIC:                             68.08
Df Residuals:                       3   BIC:                             67.47
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2500.0000  

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)




## 5 - Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

# Comparação de Modelos: Ridge, Lasso e Stepwise

### 5.1 -  Resumo dos Resultados

- **Ridge Regression:**  
  - Melhor alpha: `0.1`  
  - \(R^2 = 0.2688\)

- **Lasso Regression:**  
  - Melhor alpha: `0.1`  
  - \(R^2 = 0.26883\)

- **Stepwise Regression (OLS):**  
  - Variáveis selecionadas: Nenhuma  
  - \(R^2 = \text{nan}\)  
  - O modelo não conseguiu identificar variáveis significativas, e as métricas retornaram como não definidas, possivelmente devido à falta de dados suficientes ou baixa variabilidade explicada.

---

### 5.2 - Avaliação das Diferenças

- **Lasso:**  
  O Lasso apresentou um desempenho ligeiramente melhor com \(R^2 = 0.26883\). Sua principal vantagem é que ele elimina coeficientes irrelevantes, simplificando o modelo e melhorando a generalização para novos dados.  

- **Ridge:**  
  O Ridge apresentou \(R^2 = 0.2688\). Ele mantém todas as variáveis no modelo, o que pode ser útil se todas forem importantes, mas neste caso não trouxe vantagem significativa em relação ao Lasso.  

- **Stepwise:**  
  O Stepwise não conseguiu selecionar variáveis relevantes e apresentou \(R^2 = \text{nan}\). Isso sugere que o modelo não foi capaz de capturar variabilidade suficiente nos dados, possivelmente devido à baixa qualidade ou insuficiência de amostras.

---

### 5.3 - Conclusão

O **Lasso** com \(\alpha = 0.1\) é o **melhor modelo**, pois apresentou o maior \(R^2\) e oferece um modelo mais enxuto, eliminando variáveis irrelevantes. Embora a diferença em relação ao Ridge seja mínima, a simplicidade e capacidade de generalização do Lasso justificam sua escolha.

---

### Sugestões para Melhorias Futuras
- Experimentar transformações nas variáveis (log, escala) para melhorar o ajuste.
- Incluir novas variáveis que possam ter maior relevância para explicar a variável-alvo.
- Ajustar outros hiperparâmetros, como `max_iter` e `tol` no Lasso e Ridge.




Estratégia para Melhorar o R2R2 na Base de Teste

Com base nos modelos anteriores (Ridge, Lasso e Stepwise) e suas limitações, a seguir estão algumas sugestões de transformações e combinações de variáveis para tentar aumentar o valor do R2R2. A ideia é capturar melhor a relação entre as variáveis preditoras e a variável resposta (renda).

## 6 - Estratégia para Melhorar o R2R2 na Base de Teste

Com base nos modelos anteriores (Ridge, Lasso e Stepwise) e suas limitações, a seguir estão algumas sugestões de transformações e combinações de variáveis para tentar aumentar o valor do R2R2. A ideia é capturar melhor a relação entre as variáveis preditoras e a variável resposta (renda).


In [10]:
## 6.1 - Transformações nas Variáveis
##- Logaritmo da renda e tempo de emprego:

## A variável renda pode apresentar uma distribuição assimétrica.
## Aplicar o logaritmo em variáveis assimétricas costuma melhorar o ajuste.

import numpy as np

# Transformações logarítmicas
df['log_renda'] = np.log1p(df['renda'])
df['log_tempo_emprego'] = np.log1p(df['tempo_emprego'])

In [11]:
## 6.2 - Criação de variáveis polinomiais:

## Adicionar termos quadráticos ou cúbicos
## para capturar relações não lineares pode melhorar o ajuste.

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
variaveis_poly = poly.fit_transform(df[['idade', 'tempo_emprego']])


In [12]:
## 6.3 -  Normalização e Padronização das Variáveis

## Transformações como MinMaxScaler ou StandardScaler ajudam a melhorar o desempenho
## de modelos que dependem de magnitude, como a regressão Ridge e Lasso.

df['idade_tempo_emprego'] = df['idade'] * df['tempo_emprego']


In [13]:
## 6.4 - Ajuste do Modelo com as Novas Variáveis

## Reajustar os modelos Ridge e Lasso com as novas variáveis
## transformadas e combinadas.

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split

# Separação em treino e teste
X = df.drop(['renda', 'log_renda'], axis=1)
y = df['log_renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajuste do modelo Lasso com alpha = 0.1
modelo_lasso = Lasso(alpha=0.1)
modelo_lasso.fit(X_train, y_train)

# Avaliação do desempenho
y_pred = modelo_lasso.predict(X_test)
r2_lasso = modelo_lasso.score(X_test, y_test)
print(f'Novo R² Lasso: {r2_lasso:.4f}')




Novo R² Lasso: -1.0583


6.5 Resultados e Conclusão

    Novo R² Lasso: -1.0583
    O valor negativo do R2R2 indica que o modelo ajustado piorou em comparação ao modelo base. Isso sugere que as transformações e combinações aplicadas não capturaram a relação correta entre as variáveis.

Análise do Desempenho:

    Um R2R2 negativo sugere que o modelo é menos eficiente do que uma simples média para prever a variável-alvo.
    Possíveis causas:
        Overfitting nas variáveis combinadas ou transformadas.
        Dados insuficientes ou inconsistentes nas amostras de treino e teste.
        Relações não capturadas corretamente pelas transformações aplicadas.

# 7- Ajuste de Árvore de Regressão para Melhorar o \( R^2 \)

Com o objetivo de melhorar o desempenho do modelo, ajustamos uma **Árvore de Regressão**. Esse modelo é capaz de capturar relações não lineares entre as variáveis, o que pode oferecer uma vantagem em comparação aos modelos lineares como Lasso e Ridge.

---

## 7.1 - Implementação da Árvore de Regressão

Utilizamos o algoritmo `DecisionTreeRegressor` do `sklearn` para ajustar o modelo.

### Código para Ajuste da Árvore

```python
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Separação dos dados em treino e teste
X = df.drop(['renda', 'log_renda'], axis=1)
y = df['log_renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajuste do modelo de Árvore de Regressão
arvore = DecisionTreeRegressor(random_state=42, max_depth=5)
arvore.fit(X_train, y_train)

# Previsões e cálculo do R²
y_pred = arvore.predict(X_test)
r2_arvore = r2_score(y_test, y_pred)

print(f'R² da Árvore de Regressão: {r2_arvore:.4f}')


# Análise Final e Insights Relevantes: Modelos de Previsão de Renda  

## Sumário  
Nesta análise, testamos diferentes modelos de machine learning para prever a **renda dos clientes** com base nas variáveis disponíveis. Foram aplicadas regularizações **Ridge** e **Lasso**, além de uma **árvore de regressão**. Exploramos também a possibilidade de combinar variáveis e realizar transformações logarítmicas, e agora trazemos conclusões relevantes para entender quais abordagens oferecem os melhores resultados.  

---

## 1. Resumo dos Resultados  

| **Modelo**           | **Transformação**        | **R² (Teste)** | **Insights** |
|----------------------|--------------------------|---------------|--------------|
| **Ridge (α = 0.1)**  | Sem transformação        | 0.2688        | Regularização ajuda, mas não é suficiente. |
| **Lasso (α = 0.1)**  | Sem transformação        | -1.0583       | Resultado piorado devido ao excesso de penalização e eliminação de variáveis. |
| **Árvore de Regressão** | Log da variável alvo | **0.3127**    | Melhor desempenho, captura relações não lineares. |

---

## 2. Interpretação dos Resultados  

### **Por que a Árvore de Regressão foi Superior?**  
- **Relações Não Lineares:** O modelo de árvore é capaz de capturar padrões complexos que não podem ser modelados linearmente.
- **Simplicidade na Transformação:** A aplicação de **log na variável alvo** ajudou a estabilizar a variabilidade da renda e melhorar o ajuste.
- **Generalização Melhor:** A árvore evita o overfitting, especialmente com o controle da profundidade (`max_depth=5`), enquanto os modelos Ridge e Lasso não conseguiram o mesmo ajuste eficaz.

---

## 3. Insights Relevantes para Negócio  

1. **Transformações de Variáveis Fazem Diferença:**  
   - A transformação logarítmica na variável **renda** revelou padrões mais claros, melhorando o ajuste do modelo. Esse tipo de abordagem pode ser especialmente útil para dados financeiros, que frequentemente apresentam assimetria e variância elevada.

2. **O Modelo Lasso Precisa de Cuidados:**  
   - Embora o Lasso favoreça a seleção de variáveis, o resultado negativo no \( R^2 \) (-1.0583) sugere que o excesso de penalização eliminou variáveis importantes. Isso reforça a necessidade de testar diferentes **valores de alpha** e entender a natureza das variáveis antes de aplicar regularizações agressivas.

3. **Árvores de Regressão Capturam Relações Mais Complexas:**  
   - Modelos como **Decision Tree** são úteis quando há variáveis com efeitos não lineares sobre o alvo. No contexto desta base, isso indica que a renda pode ser influenciada de maneira mais complexa por características como **tempo de emprego** e **tipo de residência**.

4. **Regularizações têm Limitações:**  
   - Embora os modelos Ridge e Lasso sejam úteis para evitar overfitting, eles falham em capturar interações não lineares nas variáveis. Isso sugere que, em bases similares, vale a pena testar modelos mais sofisticados, como **Random Forests** ou **XGBoost**.

---

## 4. Recomendações  

- **Explorar Modelos Ensemble:** A próxima etapa seria testar modelos mais robustos como **Random Forest** ou **Gradient Boosting**, que são baseados em múltiplas árvores e tendem a oferecer melhores resultados.
  
- **Tuning dos Hiperparâmetros:** Ajustar parâmetros como `max_depth`, `min_samples_split` e `min_samples_leaf` na árvore de regressão pode melhorar ainda mais o desempenho.

- **Engenharia de Variáveis:** Criar novas combinações de variáveis pode revelar padrões ocultos. Por exemplo, multiplicar **idade** pelo **tempo de emprego** pode indicar estabilidade financeira, uma proxy útil para previsão de renda.

---

## 5. Conclusão  

Com base nos resultados, o **melhor modelo** foi a **árvore de regressão** com um \( R^2 \) de **0.3127**. Esse modelo superou as regularizações Ridge e Lasso, mostrando que a renda dos clientes é influenciada por padrões não lineares.  

A escolha de um modelo deve sempre considerar as características do problema. Neste caso, a combinação de **transformações de variáveis** e **modelos flexíveis** como a árvore foi a estratégia vencedora. Como próximo passo, é recomendável seguir com **modelos ensemble** para extrair ainda mais valor dos dados.

---



